In [1]:
from greet import greet
greet('freddy')

'Hi there, freddy'

In [43]:
animals = ['dog', 'cat', 'tiger']
pairs = enumerate(animals)
reverse = {key: index for (index, key) in pairs}
reverse

{'cat': 1, 'dog': 0, 'tiger': 2}

In [46]:
# an enumeration cannot be reused
forward = {index: key for (index, key) in pairs}
forward

{}

In [47]:
# better to inline the enumerate
forward = {index: key for (index, key) in enumerate(animals)}
forward

{0: 'dog', 1: 'cat', 2: 'tiger'}

Notes
 * an enumeration cannot be reused
